In [2]:
pip install qrcode[pil]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import qrcode

# Create a QRCode object with custom settings
qr = qrcode.QRCode(
    version=1,  # controls the size of the QR Code (1 is 21x21)
    error_correction=qrcode.constants.ERROR_CORRECT_L,  # L: 7% error correction
    box_size=10,  # size of each box in pixels
    border=4,  # thickness of the border (default is 4)
)

# Add your data
qr.add_data("Tushar")
qr.make(fit=True)

# Create the image
img = qr.make_image(fill_color="darkblue", back_color="lightyellow")

# Save it
img.save("custom_qrcode.png")
print("Custom QR code generated successfully!")

Custom QR code generated successfully!
